In [1]:
import openml
from openml.tasks import list_tasks, TaskType
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import Dynaforest
from Dynaforest import Dynatree
import logging



# Load the OpenML dataset
regression_tasks = list_tasks(task_type = TaskType.SUPERVISED_REGRESSION)
small_tasks_ids = []
for task_id, task_value in regression_tasks.items():
    if ('NumberOfInstances' in task_value.keys() and task_value['NumberOfInstances'] < 10000 and task_value['NumberOfInstances'] > 5000 and 
    'NumberOfMissingValues' in task_value.keys() and task_value['NumberOfMissingValues'] == 0 and 
    'NumberOfSymbolicFeatures' in task_value.keys() and task_value['NumberOfSymbolicFeatures'] == 0):
        small_tasks_ids.append(task_id)

In [2]:
logging.getLogger("openml.extensions.sklearn.extension").setLevel(logging.ERROR)
if not hasattr(Dynaforest, '__version__'):
    Dynaforest.__version__ = "0.0.1"  # Use an appropriate version number

task_id = small_tasks_ids[0]
task = openml.tasks.get_task(task_id)

"""Dynaforest = Dynatree(n_trees = 50, window = 5, max_depth=4, min_samples = 2, feature_subsampling_pct=0.9, 
                      delta=0.1)
RF = RandomForestRegressor(n_estimators=50, max_depth=4)
XGB = XGBRegressor(n_estimators=50, max_depth=4)

# Run your custom model on the task
run_my = openml.runs.run_model_on_task(Dynaforest, task, avoid_duplicate_runs=False)

# Run RandomForest
run_rf = openml.runs.run_model_on_task(RF, task, avoid_duplicate_runs=False)

# Run XGBoost
run_xgb = openml.runs.run_model_on_task(XGB, task, avoid_duplicate_runs=False)

dyna_mae = run_my.fold_evaluations.get("mean_absolute_error")
my_mae = sum(dyna_mae[0].values()) / len(dyna_mae)
rf_mae = run_rf.fold_evaluations.get("mean_absolute_error")
avg_rf_mae = sum(rf_mae[0].values()) / len(rf_mae)
xgb_mae = run_xgb.fold_evaluations.get("mean_absolute_error")
avg_xgb_mae = sum(xgb_mae[0].values()) / len(xgb_mae)

print(f"Task {task_id} - Mean Absolute Error:")
print(f"    MyRegressor: {my_mae:.4f}")
print(f"    RandomForest: {avg_rf_mae}")
print(f"    XGBoost: {avg_xgb_mae}")"""

'Dynaforest = Dynatree(n_trees = 50, window = 5, max_depth=4, min_samples = 2, feature_subsampling_pct=0.9, \n                      delta=0.1)\nRF = RandomForestRegressor(n_estimators=50, max_depth=4)\nXGB = XGBRegressor(n_estimators=50, max_depth=4)\n\n# Run your custom model on the task\nrun_my = openml.runs.run_model_on_task(Dynaforest, task, avoid_duplicate_runs=False)\n\n# Run RandomForest\nrun_rf = openml.runs.run_model_on_task(RF, task, avoid_duplicate_runs=False)\n\n# Run XGBoost\nrun_xgb = openml.runs.run_model_on_task(XGB, task, avoid_duplicate_runs=False)\n\ndyna_mae = run_my.fold_evaluations.get("mean_absolute_error")\nmy_mae = sum(dyna_mae[0].values()) / len(dyna_mae)\nrf_mae = run_rf.fold_evaluations.get("mean_absolute_error")\navg_rf_mae = sum(rf_mae[0].values()) / len(rf_mae)\nxgb_mae = run_xgb.fold_evaluations.get("mean_absolute_error")\navg_xgb_mae = sum(xgb_mae[0].values()) / len(xgb_mae)\n\nprint(f"Task {task_id} - Mean Absolute Error:")\nprint(f"    MyRegressor: {m

In [3]:
import numpy as np
#Read in numpy array

errors = np.load("errors.npy")
print(errors)

[[0.15475999 0.1481179  0.1556108 ]]


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

dynatree_params = {"delta": 0,
            "max_depth": 5,
            "min_samples": 1,
            "n_trees": 68,
            "window": 10, 
            'feature_subsampling_pct': 0.5}

dyna = Dynatree(**dynatree_params)
"""rf_params = {
            "max_depth": 5,
            "max_features": 0.50,
            "min_samples_leaf": 3,
            "n_estimators": 68}
dyna = RandomForestRegressor(**rf_params)
"""
X, y = task.get_X_and_y()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
dyna.fit(X_train, y_train)
train_error = mean_absolute_error(y_train, dyna.predict(X_train))
test_error = mean_absolute_error(y_test, dyna.predict(X_test))
print(f"Train error: {train_error:.4f}")
print(f"Test error: {test_error:.4f}")

This forest has a total of 2103 splits
This forest has a total of 68 trees
This forest has a total of 2103 splits
This forest has a total of 68 trees
Train error: 0.1441
Test error: 0.1544


In [7]:
trees = dyna._trees
predictions = [tree.predict(X_test) for tree in trees]
tree_biases = np.mean(predictions - y_test, axis = 1)
correlations = np.zeros((len(trees), len(trees)))
for i in range(len(trees)):
    for j in range(i, len(trees)):
        correlations[i, j] = np.corrcoef(predictions[i], predictions[j])[0, 1]
        correlations[j, i] = correlations[i, j]
print(f"Correlations: {np.mean(correlations)}")
print(f"Tree biases: {np.mean(tree_biases)}")

Correlations: 0.7743102447063231
Tree biases: 0.002009792794789265


In [99]:
trees = dyna.estimators_
predictions = [tree.predict(X_test) for tree in trees]
tree_biases = np.mean(predictions - y_test, axis = 1)
correlations = np.zeros((len(trees), len(trees)))
for i in range(len(trees)):
    for j in range(i, len(trees)):
        correlations[i, j] = np.corrcoef(predictions[i], predictions[j])[0, 1]
        correlations[j, i] = correlations[i, j]
print(f"Correlations: {np.mean(correlations)}")
print(f"Tree biases: {np.mean(tree_biases)}")

Correlations: 0.694528533687219
Tree biases: 0.003157159783847129
